>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br> 
>*Did you know you can store, visualize, and monitor whylogs profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Writing_Ranking_Performance_Metrics_to_WhyLabs)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Writing_Ranking_Performance_Metrics_to_WhyLabs) to leverage the power of whylogs and WhyLabs together!*

# Monitoring Ranking Model Performance Metrics

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/experimental/Writing_Rank_Performance_Metrics_to_WhyLabs.ipynb)

In this tutorial, we'll show how you can log ranking metrics of your ML Model with whylogs, and how to send it to your dashboard at Whylabs Platform.

## Installing whylogs

First, let's install whylogs. Since we want to write to WhyLabs, we'll install the whylabs extra. 

In [ ]:
%pip install  whylogs

In [ ]:
import pandas as pd
import whylogs as why

## Example data

We see three data formats as example data. Notice that the predictions AND targets columns can take lists or np.ndarrays of values.

Consider the predictions for each row to be a list of ranked results in decreasing rank importance order (best match is first). In the case of the `binary` example, we pass predictions with a boolean representing whether or not the prediction was successful, e.g., relevant search results.

Consider targets lists to be essentially sets of values that denote successful predictions. These can be values (matched based on equality).

In [ ]:
# Single predictions
single_df = pd.DataFrame({"raw_predictions": [["cat", "pig", "elephant"], ["horse", "donkey", "robin"],
                                          ["cow", "pig", "giraffe"], ["pig", "dolphin", "elephant"]],
                          "raw_targets": ["cat", "dog", "pig", "elephant"]})

# Binary predictions (True representing relevant results, False representing irrelevant)
binary_df = pd.DataFrame({"raw_predictions": [[True, False, True], [False, False, False],
                                          [True, True, False], [False, True, False]]})

# Multiple predictions
multiple_df = pd.DataFrame({"raw_targets": [["cat", "elephant"], ["dog", "pig"],
                                            ["pig", "cow"], ["cat", "dolphin"]],
                            "raw_predictions": [["cat", "pig", "elephant"], ["horse", "donkey", "robin"],
                                                ["cow", "pig", "giraffe"], ["pig", "dolphin", "elephant"]]})

## Log using experimental batch ranking metrics API

It's noteworthy that this API is for batch metrics -- meaning that the metrics are not meant to be merged together. This means that multiple profiles from distributed machines, Spark, or multiple uploads within the model granularity window are discouraged.

Contrastingly, non-batch metrics APIs can be merged (but one is not yet released for ranking metrics).

In [ ]:
# Log ranking metrics
from whylogs.experimental.api.logger import log_batch_ranking_metrics

results = log_batch_ranking_metrics(
    k=2,
    data=single_df,
    prediction_column="raw_predictions",
    target_column="raw_targets",
    log_full_data=True
)

# NOTE: If you've already ran why.log() on your input data, change log_full_data to False.

In [ ]:
# View results
results.view().to_pandas().T

## Send profile summaries to WhyLabs platform
Required information can be found at https://hub.whylabsapp.com under the Settings > Model and Dataset Management page.

API keys are only shown once, so you may need to create a new one and save somewhere safe.

In [ ]:
# Configure WhyLabs info, if needed
import getpass
import os

os.environ["WHYLABS_DEFAULT_ORG_ID"] = input("Enter your WhyLabs Org ID")
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = input("Enter your WhyLabs Dataset ID")
os.environ["WHYLABS_API_KEY"] =  getpass.getpass("Enter your WhyLabs API key")
print("Using API Key ID: ", os.environ["WHYLABS_API_KEY"][0:10])

In [ ]:
# Write the results to WhyLabs platform
results.writer("whylabs").write()

The ranking metrics should be available in the WhyLabs platform, under the __Performance__ tab.